In [1]:
from ift6758.data.GamesInfo import GamesInfo
import os
import json
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [2]:
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'C:\\Users\\laure\\Documents\\ift6758 - DS\\IFT6758-Project'

### Load data

In [3]:
game = GamesInfo([2017,2018], 'project/')

Loading games from 2017 located at project/games_data\2017
Loading games from 2018 located at project/games_data\2018


### Split data into regular and playoff games

In [5]:
regular_games = game.get_regular_games()
playoff_games = game.get_playoff_games()

### Get id from all of the games

In [6]:
def get_ids(games_info):
    ids = dict()
    for season, games in games_info.items():
        ids[season] = []
        for game in games:
            ids[season].append(game['gamePk'])
    return ids
regular_games_ids = get_ids(regular_games)
playoff_games_ids = get_ids(playoff_games)

## Create widgets
### Dropdown and radio buttons for season choice and game type choice (regular vs playoff)

In [7]:
season_choice=widgets.Dropdown(
    options=game.season,
    value=game.season[0],
    description='Season:',
    disabled=False,
)
game_type_choice = widgets.RadioButtons(
    options=['regular', 'playoff'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Game Type:',
    disabled=False,
)

### Slider for the choice of the game (by id)

In [8]:
if game_type_choice.value=='regular':
    games_ids_to_display = regular_games_ids[season_choice.value]
else:
    games_ids_to_display = playoff_games_ids[season_choice.value]
    
game_choice = widgets.SelectionSlider(
        options = games_ids_to_display,
        description='Game Id',
        disabled=False,
        orientation='horizontal',
        readout=True,
        continuous_update=False
    )

### Slider for the choice of the event in the chosen game

In [9]:
event_choice = widgets.IntSlider(value=0,
                                 step=1,
                                 min=0,
                                 max=len(regular_games[season_choice.value][games_ids_to_display.index(game_choice.value)]['liveData']['plays']['allPlays'])-1,
                                 description='Event Id',
                                 continuous_update=False)

### Update the game id slider when choosing a season and/or a type of game

In [10]:
def game_ids_change(sender):
    # get the right games and ids depending on season and game type
    if game_type_choice.value=='regular':
        games = regular_games_ids[season_choice.value]
    else:
        games = playoff_games_ids[season_choice.value]
        
    # change the values for the game ids slider after changing season_choice or game_type_choice
    with game_choice.hold_trait_notifications():
        game_choice.options = games
        
        
season_choice.observe(game_ids_change)
game_type_choice.observe(game_ids_change)

### Update the event id slider when the game id is modified (either because we chose a new season or game type or because we chose a new game)

In [11]:
def event_id_change(sender):
    # get the right games and ids depending on season and game type
    if game_type_choice.value=='regular':
        ids = regular_games_ids[season_choice.value]
        games = regular_games[season_choice.value]
    else:
        ids = playoff_games_ids[season_choice.value]
        games = playoff_games[season_choice.value]
    
    # change the values for the event slider after changing season_choice or game_type_choice or game_choice
    with event_choice.hold_trait_notifications():
        game = games[ids.index(game_choice.value)]
        event_choice.value = 0
        event_choice.max = len(game['liveData']['plays']['allPlays'])-1

game_choice.observe(event_id_change, names=['value', 'options'])

### Print some general information about the chosen game like the teams, who won, who lost, etc.

In [12]:
# print somw general info about the game
def show_general_info(season, game_type, game_id):
    if game_type == 'regular':
        games = regular_games[season]
        games_id = regular_games_ids[season]
    else:
        games = playoff_games[season]
        games_id = playoff_games_ids[season]
        
    # get the information to print    
    game = games[games_id.index(game_id)]
    home = game['gameData']['teams']['home']['abbreviation'] # home team
    away = game['gameData']['teams']['away']['abbreviation'] # away team
    goals = game['liveData']['plays']['currentPlay']['about']['goals'] # goals from both teams
    sog = game['liveData']['linescore']['teams'] # shots on goal by both teams
    info = [['', 'Home', 'Away'], 
            ['Teams', home, away], 
            ['Goals', goals['home'], goals['away']],
            ['SoG', sog['home']['shotsOnGoal'], sog['away']['shotsOnGoal']]]
    
    # print everything
    print(game['gameData']['datetime']['dateTime'])
    print('Game ID: {} {} (Home) {} (Away)'.format(str(game_id)[-4:], home, away))
    for row in info:
        print('\t{: <10} {: <10} {: <10}'.format(*row))
        
date = widgets.interactive_output(show_general_info, {'season': season_choice, 'game_type': game_type_choice, 'game_id': game_choice})

### Load the ice rink picture

In [ ]:
ice_rink = plt.imread('project/figures/nhl_rink.png')

### Show the event on the ice rink with the coordinates and print information about the event

In [14]:
# show ice rink with the players of the event on it
def show_event(season, game_type, game_id, event_id):
    if game_type == 'regular':
        games = regular_games[season]
        games_id = regular_games_ids[season]
    else:
        games = playoff_games[season]
        games_id = playoff_games_ids[season]
    
    # get all of the information necessary to make the plot
    game = games[games_id.index(game_id)]
    home = game['gameData']['teams']['home']['abbreviation'] # the home team
    away = game['gameData']['teams']['away']['abbreviation'] # the away team
    events = game['liveData']['plays']['allPlays']
    event = events[event_id] # the event information
    description = event['result']['description'] # description of the event
    time = event['about']['periodTime'] # time when the event happened
    period = event['about']['period'] # which period 
    coordinates = event['coordinates'] # the coordinate of the event
    left = home if game['liveData']['linescore']['periods'][period-1]['home']['rinkSide'] == 'left' else away # team on left side of rink
    right = home if game['liveData']['linescore']['periods'][period-1]['home']['rinkSide'] == 'right' else away # team on right side of rink
    
    # create ice rink image
    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.imshow(ice_rink, extent=[-100,100,-42.5,42.5])
    
    # plot where the event happened if there are coordinates
    if len(coordinates)>0:
        ax.plot(coordinates['x'],coordinates['y'], 'go', markersize=8)
    
    ax.set_facecolor('black')
    ax.set_xlabel('feet')
    ax.set_ylabel('feet')
    ax.set_title('{} \n{} P-{}'.format(description, time, period))
    plt.text(-65, 45, left)
    plt.text(50, 45, right)
    plt.show()
    
    # print the information about the event
    print(json.dumps(event, indent=4, sort_keys=True))
    
rink = widgets.interactive_output(show_event, {'season': season_choice, 'game_type': game_type_choice, 'game_id': game_choice, 'event_id': event_choice})

## Applet

In [16]:
widgets.VBox([season_choice, game_type_choice, game_choice, date, event_choice, rink])